In [12]:
import pandas as pd
import numpy as np

In [33]:
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

set_config(display='diagram')

In [18]:
df = pd.DataFrame(data=[['Carlo', 'Birra',    100, 200, 0.06, 199],
                       ['Barbara', 'Bartolo', 200, 200, 0.04, 162],
                       ['Carlo', 'Bolle',     100, 200, 0.001, 202]], 
                 columns=("nome","cognome", "var1", "var2", "var3", "var4"))
df

,nome,cognome,var1,var2,var3,var4
0,Carlo,Birra,100,200,0.060,199
1,Barbara,Bartolo,200,200,0.040,162
2,Carlo,Bolle,100,200,0.001,202


In [34]:
num_proc = make_pipeline(SimpleImputer(strategy='median'), MinMaxScaler())
num_proc

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('minmaxscaler', MinMaxScaler())])

In [35]:
cat_proc = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore'))
cat_proc

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

In [36]:
preprocessor = make_column_transformer((cat_proc, ('nome', 'cognome')),
                                       (num_proc, ('var1', 'var2')),
                                       ('drop', ('var3')),
                                        remainder='passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ('nome', 'cognome')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ('var1', 'var2')),
                                ('drop', 'drop', 'var3')])

In [37]:
df1 = preprocessor.fit_transform(df)
pd.DataFrame(df1)

,0,1,2,3,4,5,6,7
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,199.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,162.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,202.0


In [8]:
clf = make_pipeline(preprocessor, LogisticRegression())
clf

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ('feat1', 'feat3')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ('feat0', 'feat2'))])),
                ('logisticregression', LogisticRegression())])